# Scraping playlists and getting audio features

In [1]:
import requests
import pandas as pd
from time import sleep
import random
from time import sleep
from random import randint
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import datetime
import time
import joblib

secrets_file = open(r"C:\Users\luana\secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['secret']))

In [9]:
# This function takes a playlist id and creates a dataframe with all tracks and their features

def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(random.uniform(0.5,1))
   

    #creating df with needed columns
    playlist_df = pd.DataFrame(columns=['song','artist','popularity','danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                    'type','id','uri','track_href','analysis_url','duration_ms','time_signature'])
    
    for i in range(len(tracks)):
        sleep(random.uniform(0.5,1))
        
        try:
        
            song_id = tracks[i]['track']['id']

            # Song name, artists and popularity obtained directly with request

            song = sp.track(song_id)['name']

            # Getting up to three different artists for each song
            artist_1 = sp.track(song_id)['artists'][0]['name']
            artist_2 = ''
            try:
                artist_2 = sp.track(song_id)['artists'][1]['name']
                artist_2= " & "+artist_2
            except:
                pass
            artist_3 = ''
            try:
                artist_3 = sp.track(song_id)['artists'][2]['name']
                artist_3= " & "+artist_3
            except:
                pass
            artists = (artist_1+artist_2+artist_3)

            popularity = sp.track(song_id)['popularity']

            track_dict = {'song':song, 'artist':artists, 'popularity':popularity}
            df_track = pd.DataFrame([track_dict])

            # Now the audio features
            features_dict = sp.audio_features(song_id)
            df_features = pd.DataFrame(features_dict)

            # Complete df for one song
            song_df = pd.concat([df_track, df_features], axis=1)

            # Complete df with all songs
            playlist_df = pd.concat([playlist_df, song_df],axis=0)

            
        except:
            pass    

    return playlist_df

In [12]:
# This function will take a list of queries and get all songs from all playlists

def search_export_playlists(queries_list):
    songs_df = pd.DataFrame(columns=['song','artist','popularity','danceability','energy','key','loudness','mode',
                                    'speechiness','acousticness','instrumentalness','liveness','valence','tempo',
                                    'type','id','uri','track_href','analysis_url','duration_ms','time_signature'])
    
    for i in range (len(queries_list)):
        sleep(random.uniform(0.5,1))
        try:
                     
            results = sp.search(q=queries_list[i], type='playlist', limit=1)
            playlist_id = results['playlists']['items'][0]['id']

            playlist_df = get_playlist_tracks(playlist_id)

            songs_df = pd.concat([songs_df, playlist_df], axis=0)

            print(queries_list[i], ' done')
            
            # Progress report every 25%
            if i == round((len(queries_list)*0.25)):
                print("25% complete")
            elif i == round((len(queries_list)*0.5)):
                print("50% complete")
            elif i == round((len(queries_list)*0.75)):
                print('75% complete')
            else:
                pass
                  
        except:
            pass
        
   
    return songs_df

In [10]:
# %%time
# queries = ['Jazz']
# test_df = search_export_playlists(queries)
# test_df

Jazz  done
25% complete
CPU times: total: 12.2 s
Wall time: 8min 14s


,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Without A Song,Meadow Park,67,0.475,0.138,5,-17.099,0,0.0424,0.965,...,0.0971,0.123,118.338,audio_features,1nsMASRDWwUwlBMmOuh80d,spotify:track:1nsMASRDWwUwlBMmOuh80d,https://api.spotify.com/v1/tracks/1nsMASRDWwUw...,https://api.spotify.com/v1/audio-analysis/1nsM...,236785,4
0,Sometime Ago,Madeleine West,66,0.538,0.14,9,-17.368,0,0.0393,0.937,...,0.324,0.195,86.259,audio_features,6HZgxhesyOy8jkTiAFfQmR,spotify:track:6HZgxhesyOy8jkTiAFfQmR,https://api.spotify.com/v1/tracks/6HZgxhesyOy8...,https://api.spotify.com/v1/audio-analysis/6HZg...,160919,3
0,Prelude To A Kiss,Al Fowler Trio,64,0.629,0.152,9,-18.659,0,0.04,0.967,...,0.109,0.0966,116.833,audio_features,4Csbgu7LMDGPLjhPgHfQmZ,spotify:track:4Csbgu7LMDGPLjhPgHfQmZ,https://api.spotify.com/v1/tracks/4Csbgu7LMDGP...,https://api.spotify.com/v1/audio-analysis/4Csb...,275743,4
0,Autumn In New York,Mayfare Trio,63,0.546,0.152,7,-17.08,0,0.0368,0.958,...,0.3,0.144,124.562,audio_features,62FrtiVepyS9UGfcTPhu7W,spotify:track:62FrtiVepyS9UGfcTPhu7W,https://api.spotify.com/v1/tracks/62FrtiVepyS9...,https://api.spotify.com/v1/audio-analysis/62Fr...,259680,4
0,Love To Be Happy,Magpie Jazz Trio,5,0.466,0.0718,6,-19.581,1,0.0336,0.922,...,0.11,0.214,105.252,audio_features,3wMe1lySKxJeH9iMWXDqgx,spotify:track:3wMe1lySKxJeH9iMWXDqgx,https://api.spotify.com/v1/tracks/3wMe1lySKxJe...,https://api.spotify.com/v1/audio-analysis/3wMe...,195276,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,You Don't Know What Love Is,David Hazeltine & Peter Washington & Joe Farns...,47,0.453,0.155,1,-23.248,1,0.0358,0.949,...,0.101,0.208,117.986,audio_features,6rLVGBoRQecxIEjucWATyT,spotify:track:6rLVGBoRQecxIEjucWATyT,https://api.spotify.com/v1/tracks/6rLVGBoRQecx...,https://api.spotify.com/v1/audio-analysis/6rLV...,428387,4
0,Quite Late,Turning Trio,47,0.566,0.22,8,-15.091,1,0.0328,0.86,...,0.115,0.147,119.33,audio_features,5PzrM2NUo81JLdzNILxMLd,spotify:track:5PzrM2NUo81JLdzNILxMLd,https://api.spotify.com/v1/tracks/5PzrM2NUo81J...,https://api.spotify.com/v1/audio-analysis/5Pzr...,176539,3
0,Wichita Lineman,Alan Pasqua,48,0.594,0.0571,5,-23.622,1,0.0441,0.936,...,0.14,0.259,113.163,audio_features,4ytxg5HNzbG1Y8CdZLtQj2,spotify:track:4ytxg5HNzbG1Y8CdZLtQj2,https://api.spotify.com/v1/tracks/4ytxg5HNzbG1...,https://api.spotify.com/v1/audio-analysis/4ytx...,358453,4
0,Absence,Neil Chapman Green,47,0.489,0.128,3,-18.782,1,0.0316,0.925,...,0.101,0.148,96.816,audio_features,0q243837xX1qJmXuItLJHd,spotify:track:0q243837xX1qJmXuItLJHd,https://api.spotify.com/v1/tracks/0q243837xX1q...,https://api.spotify.com/v1/audio-analysis/0q24...,191565,4


In [15]:
# %%time
# queries = ['Jazz']
# test_df = search_export_playlists(queries)
# test_df

#test after reducing sleeping time between songs


Jazz  done
25% complete
CPU times: total: 9.86 s
Wall time: 5min 3s


,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Without A Song,Meadow Park,67,0.475,0.138,5,-17.099,0,0.0424,0.965,...,0.0971,0.123,118.338,audio_features,1nsMASRDWwUwlBMmOuh80d,spotify:track:1nsMASRDWwUwlBMmOuh80d,https://api.spotify.com/v1/tracks/1nsMASRDWwUw...,https://api.spotify.com/v1/audio-analysis/1nsM...,236785,4
0,Sometime Ago,Madeleine West,66,0.538,0.14,9,-17.368,0,0.0393,0.937,...,0.324,0.195,86.259,audio_features,6HZgxhesyOy8jkTiAFfQmR,spotify:track:6HZgxhesyOy8jkTiAFfQmR,https://api.spotify.com/v1/tracks/6HZgxhesyOy8...,https://api.spotify.com/v1/audio-analysis/6HZg...,160919,3
0,Prelude To A Kiss,Al Fowler Trio,64,0.629,0.152,9,-18.659,0,0.04,0.967,...,0.109,0.0966,116.833,audio_features,4Csbgu7LMDGPLjhPgHfQmZ,spotify:track:4Csbgu7LMDGPLjhPgHfQmZ,https://api.spotify.com/v1/tracks/4Csbgu7LMDGP...,https://api.spotify.com/v1/audio-analysis/4Csb...,275743,4
0,Autumn In New York,Mayfare Trio,63,0.546,0.152,7,-17.08,0,0.0368,0.958,...,0.3,0.144,124.562,audio_features,62FrtiVepyS9UGfcTPhu7W,spotify:track:62FrtiVepyS9UGfcTPhu7W,https://api.spotify.com/v1/tracks/62FrtiVepyS9...,https://api.spotify.com/v1/audio-analysis/62Fr...,259680,4
0,Love To Be Happy,Magpie Jazz Trio,5,0.466,0.0718,6,-19.581,1,0.0336,0.922,...,0.11,0.214,105.252,audio_features,3wMe1lySKxJeH9iMWXDqgx,spotify:track:3wMe1lySKxJeH9iMWXDqgx,https://api.spotify.com/v1/tracks/3wMe1lySKxJe...,https://api.spotify.com/v1/audio-analysis/3wMe...,195276,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,You Don't Know What Love Is,David Hazeltine & Peter Washington & Joe Farns...,47,0.453,0.155,1,-23.248,1,0.0358,0.949,...,0.101,0.208,117.986,audio_features,6rLVGBoRQecxIEjucWATyT,spotify:track:6rLVGBoRQecxIEjucWATyT,https://api.spotify.com/v1/tracks/6rLVGBoRQecx...,https://api.spotify.com/v1/audio-analysis/6rLV...,428387,4
0,Quite Late,Turning Trio,47,0.566,0.22,8,-15.091,1,0.0328,0.86,...,0.115,0.147,119.33,audio_features,5PzrM2NUo81JLdzNILxMLd,spotify:track:5PzrM2NUo81JLdzNILxMLd,https://api.spotify.com/v1/tracks/5PzrM2NUo81J...,https://api.spotify.com/v1/audio-analysis/5Pzr...,176539,3
0,Wichita Lineman,Alan Pasqua,48,0.594,0.0571,5,-23.622,1,0.0441,0.936,...,0.14,0.259,113.163,audio_features,4ytxg5HNzbG1Y8CdZLtQj2,spotify:track:4ytxg5HNzbG1Y8CdZLtQj2,https://api.spotify.com/v1/tracks/4ytxg5HNzbG1...,https://api.spotify.com/v1/audio-analysis/4ytx...,358453,4
0,Absence,Neil Chapman Green,47,0.489,0.128,3,-18.782,1,0.0316,0.925,...,0.101,0.148,96.816,audio_features,0q243837xX1qJmXuItLJHd,spotify:track:0q243837xX1qJmXuItLJHd,https://api.spotify.com/v1/tracks/0q243837xX1q...,https://api.spotify.com/v1/audio-analysis/0q24...,191565,4


In [23]:
# %%time
# queries = ['Pop', 'Rock and Roll', 'Jazz', 'Blues', 'R&B','Hip hop', 'Reggae', 'Electronic', 'House', 'Techno', 'K Pop',
#            'Latin', 'Indie', 'Metal', 'Country']
# songs_genre_1 = search_export_playlists(queries)

Pop  done
Rock and Roll  done
Reggae  done
Electronic  done
House  done
50% complete
Techno  done
K Pop  done
Latin  done
75% complete
Indie  done
Metal  done
Country  done
CPU times: total: 35.5 s
Wall time: 18min 43s


In [25]:
# songs_genre_1.shape

(1231, 21)

In [24]:
# songs_genre_1.to_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\songs_genre_1.csv', index = False, header=True)

In [4]:
# %%time
# queries = ['Rap', 'Dance', 'Gospel', 'Classical', 'Alternative', 'Punk', 'Trap',
#            'Dance', 'Bossa Nova', 'Reggaeton', 'Samba', 'Salsa', 'Soul', 'Funk', 'Folk']
# songs_genre_2 = search_export_playlists(queries)

Rap  done
Dance  done
Gospel  done
Classical  done
Alternative  done
25% complete
Punk  done
Trap  done
Dance  done
Bossa Nova  done
50% complete
Reggaeton  done
Samba  done
Salsa  done
75% complete
Soul  done
Funk  done
Folk  done
CPU times: total: 40.5 s
Wall time: 19min 11s


In [5]:
# songs_genre_2

,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,California Breeze,Lil Baby,0,0.738,0.666,5,-6.97,0,0.455,0.00361,...,0.107,0.215,161.734,audio_features,6ug9fUi5oLLgQgOF1G8WkM,spotify:track:6ug9fUi5oLLgQgOF1G8WkM,https://api.spotify.com/v1/tracks/6ug9fUi5oLLg...,https://api.spotify.com/v1/audio-analysis/6ug9...,177789,4
0,Bye Bye,Marshmello & Juice WRLD,0,0.645,0.525,3,-5.399,0,0.0355,0.0574,...,0.505,0.238,83.018,audio_features,6XO8RlYuJCiI0v3IA48FeJ,spotify:track:6XO8RlYuJCiI0v3IA48FeJ,https://api.spotify.com/v1/tracks/6XO8RlYuJCiI...,https://api.spotify.com/v1/audio-analysis/6XO8...,108499,4
0,Poland,Lil Yachty,75,0.695,0.555,5,-8.5,0,0.0509,0.141,...,0.11,0.26,150.024,audio_features,74loibzxXRL875X20kenvk,spotify:track:74loibzxXRL875X20kenvk,https://api.spotify.com/v1/tracks/74loibzxXRL8...,https://api.spotify.com/v1/audio-analysis/74lo...,83200,4
0,Backstage Passes (feat. Jack Harlow),EST Gee & Jack Harlow,77,0.855,0.478,1,-8.31,1,0.345,0.145,...,0.101,0.536,153.074,audio_features,2BSl2gSFHdzJAMus74qzNj,spotify:track:2BSl2gSFHdzJAMus74qzNj,https://api.spotify.com/v1/tracks/2BSl2gSFHdzJ...,https://api.spotify.com/v1/audio-analysis/2BSl...,158431,4
0,Blow,Moneybagg Yo,76,0.815,0.742,11,-4.896,0,0.245,0.0579,...,0.345,0.584,137.036,audio_features,2g3qKijfeHfDyM7Fmaoi5M,spotify:track:2g3qKijfeHfDyM7Fmaoi5M,https://api.spotify.com/v1/tracks/2g3qKijfeHfD...,https://api.spotify.com/v1/audio-analysis/2g3q...,182533,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Parasite,Nick Drake,48,0.61,0.353,11,-9.289,1,0.0296,0.76,...,0.102,0.237,131.257,audio_features,2tlhnUxHtYDqu0i9KIraUu,spotify:track:2tlhnUxHtYDqu0i9KIraUu,https://api.spotify.com/v1/tracks/2tlhnUxHtYDq...,https://api.spotify.com/v1/audio-analysis/2tlh...,214026,4
0,Amsterdam,Wild Rivers,65,0.637,0.482,1,-9.427,1,0.0284,0.217,...,0.0904,0.593,140.091,audio_features,52BtapS0Wy7LM8R6QvJRpp,spotify:track:52BtapS0Wy7LM8R6QvJRpp,https://api.spotify.com/v1/tracks/52BtapS0Wy7L...,https://api.spotify.com/v1/audio-analysis/52Bt...,213920,4
0,Boardwalks - Remastered 2015/ Bonus Track,Little May,61,0.662,0.384,6,-12.634,1,0.0281,0.775,...,0.117,0.261,128.992,audio_features,23Zlb66N7xcIWbpt2C1bsi,spotify:track:23Zlb66N7xcIWbpt2C1bsi,https://api.spotify.com/v1/tracks/23Zlb66N7xcI...,https://api.spotify.com/v1/audio-analysis/23Zl...,199360,4
0,Secret - Acoustic,Angel Snow,57,0.599,0.295,3,-11.548,1,0.0264,0.655,...,0.102,0.145,112.579,audio_features,7khUiiyqFAluYRsw1ZKcWS,spotify:track:7khUiiyqFAluYRsw1ZKcWS,https://api.spotify.com/v1/tracks/7khUiiyqFAlu...,https://api.spotify.com/v1/audio-analysis/7khU...,190000,4


In [5]:
# songs_genre_2.to_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\songs_genre_2.csv', index = False, header=True)

In [1]:
# %%time
# queries = ['MPB', 'J pop', 'Lambada', 'Heavy Metal', 'Alternative', 'Ambient', 'Swing', 'Trance', 'Instrumental',
#           'Acoustic', 'New Wave', 'Opera']
# songs_genre_3 = search_export_playlists(queries)

In [ ]:
# songs_genre_3.to_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\songs_genre_3.csv', index = False, header=True)
# songs_genre_3

In [6]:
songs_genre_1 = pd.read_csv('songs_genre_1.csv')
songs_genre_2 = pd.read_csv('songs_genre_2.csv')
songs_genre_3 = pd.read_csv('songs_genre_3.csv')
songs_genre = pd.concat([songs_genre_1,songs_genre_2,songs_genre_3], axis=0)
songs_genre.to_csv(r'C:\Users\luana\IH-Labs\lab-web-scraping-multiple-pages\songs_genre.csv', index = False, header=True)
songs_genre

,song,artist,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,As It Was,Harry Styles,93,0.520,0.731,6,-5.338,0,0.0557,0.342000,...,0.3110,0.662,173.930,audio_features,4Dvkj6JhhA12EX05fT7y2e,spotify:track:4Dvkj6JhhA12EX05fT7y2e,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...,167303,4
1,Perfect,Ed Sheeran,88,0.599,0.448,8,-6.312,1,0.0232,0.163000,...,0.1060,0.168,95.050,audio_features,0tgVpDi06FyKpA1z0VMD4v,spotify:track:0tgVpDi06FyKpA1z0VMD4v,https://api.spotify.com/v1/tracks/0tgVpDi06FyK...,https://api.spotify.com/v1/audio-analysis/0tgV...,263400,3
2,STAR WALKIN' (League of Legends Worlds Anthem),Lil Nas X,90,0.637,0.715,2,-4.971,0,0.0455,0.148000,...,0.0892,0.308,141.872,audio_features,38T0tPVZHcPZyhtOcCP7pF,spotify:track:38T0tPVZHcPZyhtOcCP7pF,https://api.spotify.com/v1/tracks/38T0tPVZHcPZ...,https://api.spotify.com/v1/audio-analysis/38T0...,210576,4
3,Treat You Better,Shawn Mendes,84,0.444,0.819,10,-4.078,0,0.3410,0.106000,...,0.1070,0.747,82.695,audio_features,3QGsuHI8jO1Rx4JWLUh9jd,spotify:track:3QGsuHI8jO1Rx4JWLUh9jd,https://api.spotify.com/v1/tracks/3QGsuHI8jO1R...,https://api.spotify.com/v1/audio-analysis/3QGs...,187973,4
4,Dark Horse,Katy Perry & Juicy J,77,0.647,0.585,6,-6.123,1,0.0512,0.003140,...,0.1650,0.353,131.934,audio_features,4jbmgIyjGoXjY01XxatOx6,spotify:track:4jbmgIyjGoXjY01XxatOx6,https://api.spotify.com/v1/tracks/4jbmgIyjGoXj...,https://api.spotify.com/v1/audio-analysis/4jbm...,215672,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803,Starter,Marco Lys & Tube & Berger,45,0.719,0.954,1,-5.367,1,0.0656,0.000781,...,0.1000,0.614,124.037,audio_features,4oFlRoHqdnjfPm0tUUIvAq,spotify:track:4oFlRoHqdnjfPm0tUUIvAq,https://api.spotify.com/v1/tracks/4oFlRoHqdnjf...,https://api.spotify.com/v1/audio-analysis/4oFl...,178142,4
1804,Chromatic,Anti Up,61,0.607,0.914,0,-5.020,0,0.0547,0.000038,...,0.1650,0.781,128.032,audio_features,3FcKnF4nEwGxlwdAmRIogh,spotify:track:3FcKnF4nEwGxlwdAmRIogh,https://api.spotify.com/v1/tracks/3FcKnF4nEwGx...,https://api.spotify.com/v1/audio-analysis/3FcK...,202524,4
1805,I'm The Joker (feat. DJ Supreme),Darius Syrossian & DJ Supreme,43,0.802,0.889,10,-6.715,0,0.1100,0.005980,...,0.5540,0.883,129.025,audio_features,3SHFOtqLr93qSbz4OyqJYU,spotify:track:3SHFOtqLr93qSbz4OyqJYU,https://api.spotify.com/v1/tracks/3SHFOtqLr93q...,https://api.spotify.com/v1/audio-analysis/3SHF...,156299,4
1806,Slow Motion,Volaris,48,0.497,0.986,3,-5.892,1,0.0404,0.000337,...,0.1920,0.163,126.980,audio_features,6KCFVYOU3FOJcffbqkoBlx,spotify:track:6KCFVYOU3FOJcffbqkoBlx,https://api.spotify.com/v1/tracks/6KCFVYOU3FOJ...,https://api.spotify.com/v1/audio-analysis/6KCF...,204094,4


In [ ]:
# I cant scrape more information from Spotify right now so I will add a Kaggle dataframe to increase the number of songs